In [ ]:
!pip install pandas

In [ ]:
!pip install sklearn

In [ ]:
!pip install matplotlib

In [ ]:
!pip install librosa==0.8.0

In [ ]:
!apt-get update

In [ ]:
!apt-get install libsndfile-dev -y

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import svm
from sklearn import metrics
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
from pathlib import Path
import tensorflow as tf
import shutil
import librosa
from tqdm import tqdm_notebook
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

SVM Approach

In [ ]:
data = pd.read_csv('dataset_train.csv')
data = data.drop(['filename'],axis=1)
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y_train = encoder.fit_transform(genre_list)
scaler = StandardScaler()
X_train = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

data = pd.read_csv('dataset_dev.csv')
data = data.drop(['filename'],axis=1)
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y_dev = encoder.fit_transform(genre_list)
scaler = StandardScaler()
X_dev = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

data = pd.read_csv('dataset_eval.csv')
data = data.drop(['filename'],axis=1)
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y_eval = encoder.fit_transform(genre_list)
scaler = StandardScaler()
X_eval = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [ ]:
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_train)
print('Train')
print("Accuracy:",metrics.accuracy_score(y_train, y_pred))
print()
y_pred = clf.predict(X_dev)
print('Dev')
print("Accuracy:",metrics.accuracy_score(y_dev, y_pred))
print()
y_pred = clf.predict(X_eval)
print('Eval')
print("Accuracy:",metrics.accuracy_score(y_eval, y_pred))
print()

In [ ]:
import time

training_start = time.perf_counter()

X_test= X_eval
y_test= y_eval

clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)
training_end = time.perf_counter()

prediction_start = time.perf_counter()
preds = clf.predict(X_test)
prediction_end = time.perf_counter()

acc_clf = (preds == y_test).sum().astype(float) / len(preds)*100
clf_train_time = training_end-training_start
clf_prediction_time = prediction_end-prediction_start
print("Scikit-Learn's Random Forest Classifier's prediction accuracy is: %3.2f" % (acc_clf))
print("Time consumed for training: %4.3f seconds" % (clf_train_time))
print("Time consumed for prediction: %6.5f seconds" % (clf_prediction_time))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

df_rfc = pd.DataFrame({'Class': np.unique(preds) ,
             'Precision': precision_score(y_test, preds, average=None).round(4)*100,
              'Recall': recall_score(y_test, preds, average=None).round(4)*100,
              'F-Score': f1_score(y_test, preds, average=None).round(4)*100
             })

display(df_rfc)

pre_rfc=precision_score(y_test, preds, average='weighted')*100
rec_rfc=recall_score(y_test, preds, average='weighted')*100
f1_rfc=f1_score(y_test, preds, average='weighted')*100
acc_rfc=accuracy_score(y_test, preds)*100

print('Metrics:\n')
print('Precision: %.2f%%' % (pre_rfc))
print('Recall: %.2f%%' % (rec_rfc))
print('F_score: %.2f%%' % (f1_rfc))
print('Accuracy: %.2f%%'% (acc_rfc))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, preds))
print()
print("Classification Report")
print(classification_report(y_test, preds))

In [ ]:
#Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=1000)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_train)
print('Train')
print("Accuracy:",metrics.accuracy_score(y_train, y_pred))
print()
y_pred = rfc.predict(X_dev)
print('Dev')
print("Accuracy:",metrics.accuracy_score(y_dev, y_pred))
print()
y_pred = rfc.predict(X_eval)
print('Eval')
print("Accuracy:",metrics.accuracy_score(y_eval, y_pred))
print()

In [ ]:
import time

training_start = time.perf_counter()

X_test= X_eval
y_test= y_eval

rfc = RandomForestClassifier(n_estimators=100)
# Model Fit and Predictions
rfc.fit(X_train, y_train)
training_end = time.perf_counter()

prediction_start = time.perf_counter()
preds = rfc.predict(X_test)
prediction_end = time.perf_counter()

acc_rfc = (preds == y_test).sum().astype(float) / len(preds)*100
rfc_train_time = training_end-training_start
rfc_prediction_time = prediction_end-prediction_start
print("Scikit-Learn's Random Forest Classifier's prediction accuracy is: %3.2f" % (acc_rfc))
print("Time consumed for training: %4.3f seconds" % (rfc_train_time))
print("Time consumed for prediction: %6.5f seconds" % (rfc_prediction_time))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

df_rfc = pd.DataFrame({'Class': np.unique(preds) ,
             'Precision': precision_score(y_test, preds, average=None).round(4)*100,
              'Recall': recall_score(y_test, preds, average=None).round(4)*100,
              'F-Score': f1_score(y_test, preds, average=None).round(4)*100
             })

display(df_rfc)

pre_rfc=precision_score(y_test, preds, average='weighted')*100
rec_rfc=recall_score(y_test, preds, average='weighted')*100
f1_rfc=f1_score(y_test, preds, average='weighted')*100
acc_rfc=accuracy_score(y_test, preds)*100

print('Metrics:\n')
print('Precision: %.2f%%' % (pre_rfc))
print('Recall: %.2f%%' % (rec_rfc))
print('F_score: %.2f%%' % (f1_rfc))
print('Accuracy: %.2f%%'% (acc_rfc))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, preds))
print()
print("Classification Report")
print(classification_report(y_test, preds))

In [ ]:
#kNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

In [ ]:
training_start = time.perf_counter()
knn.fit(X_train, y_train)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = knn.predict(X_test)
prediction_end = time.perf_counter()
acc_knn = (preds == y_test).sum().astype(float) / len(preds)*100
knn_train_time = training_end-training_start
knn_prediction_time = prediction_end-prediction_start
print("Scikit-Learn's K Nearest Neighbors Classifier's prediction accuracy is: %3.2f" % (acc_knn))
print("Time consumed for training: %4.3f seconds" % (knn_train_time))
print("Time consumed for prediction: %6.5f seconds" % (knn_prediction_time))

In [ ]:
df_knn = pd.DataFrame({'Class': np.unique(preds) ,
             'Precision': precision_score(y_test, preds, average=None).round(4)*100,
              'Recall': recall_score(y_test, preds, average=None).round(4)*100,
              'F-Score': f1_score(y_test, preds, average=None).round(4)*100
             })

display(df_knn)

pre_knn=precision_score(y_test, preds, average='weighted')*100
rec_knn=recall_score(y_test, preds, average='weighted')*100
f1_knn=f1_score(y_test, preds, average='weighted')*100
acc_knn=accuracy_score(y_test, preds)*100

print('Metrics:\n')
print('Precision: %.2f%%' % (pre_knn))
print('Recall: %.2f%%' % (rec_knn))
print('F_score: %.2f%%' % (f1_knn))
print('Accuracy: %.2f%%'% (acc_knn))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, preds))
print()
print("Classification Report")
print(classification_report(y_test, preds))

In [ ]:
#XGBOOST

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb= XGBClassifier(n_estimators=100)

In [ ]:
training_start = time.perf_counter()
xgb.fit(X_train, y_train)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = xgb.predict(X_test)
prediction_end = time.perf_counter()
acc_xgb = (preds == y_test).sum().astype(float) / len(preds)*100
xgb_train_time = training_end-training_start
xgb_prediction_time = prediction_end-prediction_start
print("Scikit-Learn's XGBOOST prediction accuracy is: %3.2f" % (acc_xgb))
print("Time consumed for training: %4.3f seconds" % (xgb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (xgb_prediction_time))

In [ ]:
df_xgb = pd.DataFrame({'Class': np.unique(preds) ,
             'Precision': precision_score(y_test, preds, average=None).round(4)*100,
              'Recall': recall_score(y_test, preds, average=None).round(4)*100,
              'F-Score': f1_score(y_test, preds, average=None).round(4)*100
             })

display(df_xgb)

pre_xgb=precision_score(y_test, preds, average='weighted')*100
rec_xgb=recall_score(y_test, preds, average='weighted')*100
f1_xgb=f1_score(y_test, preds, average='weighted')*100
acc_xgb=accuracy_score(y_test, preds)*100

print('Metrics:\n')
print('Precision: %.2f%%' % (pre_xgb))
print('Recall: %.2f%%' % (rec_xgb))
print('F_score: %.2f%%' % (f1_xgb))
print('Accuracy: %.2f%%'% (acc_xgb))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, preds))
print()
print("Classification Report")
print(classification_report(y_test, preds))

In [ ]:
!pip install lightgbm

In [ ]:
###LGBM


from lightgbm import LGBMClassifier

lgb = LGBMClassifier(
    objective='binary',
    boosting='gbdt',
    learning_rate = 0.05,
    max_depth = 8,
    num_leaves = 80,
    n_estimators = 200,
    bagging_fraction = 0.8,
    feature_fraction = 0.9)
    #reg_alpha = 0.2,
    #reg_lambda = 0.4)

In [ ]:
training_start = time.perf_counter()
lgb.fit(X_train, y_train)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = lgb.predict(X_test)
prediction_end = time.perf_counter()
acc_lgb = (preds == y_test).sum().astype(float) / len(preds)*100
lgb_train_time = training_end-training_start
lgb_prediction_time = prediction_end-prediction_start
print("Scikit-Learn's LGBM prediction accuracy is: %3.2f" % (acc_lgb))
print("Time consumed for training: %4.3f seconds" % (lgb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (lgb_prediction_time))

In [ ]:
df_lgb = pd.DataFrame({'Class': np.unique(preds) ,
             'Precision': precision_score(y_test, preds, average=None).round(4)*100,
              'Recall': recall_score(y_test, preds, average=None).round(4)*100,
              'F-Score': f1_score(y_test, preds, average=None).round(4)*100
             })

display(df_lgb)

pre_lgb=precision_score(y_test, preds, average='weighted')*100
rec_lgb=recall_score(y_test, preds, average='weighted')*100
f1_lgb=f1_score(y_test, preds, average='weighted')*100
acc_lgb=accuracy_score(y_test, preds)*100

print('Metrics:\n')
print('Precision: %.2f%%' % (pre_lgb))
print('Recall: %.2f%%' % (rec_lgb))
print('F_score: %.2f%%' % (f1_lgb))
print('Accuracy: %.2f%%'% (acc_lgb))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, preds))
print()
print("Classification Report")
print(classification_report(y_test, preds))

CNN Approaches

In [ ]:
def lstm(input_shape = (None, 37), lr = 1e-3):
    inputs = layers.Input(input_shape)
    X = layers.LSTM(64, return_sequences=True)(inputs)
    X = layers.LSTM(64, return_sequences=True)(X)
    X = layers.Dropout(rate=0.5)(X)
    X = layers.LSTM(128, return_sequences=True)(X)
    X = layers.LSTM(128, return_sequences=True)(X)
    X = layers.Dropout(rate=0.5)(X)
    X = layers.LSTM(256, return_sequences=False)(X)
    X = layers.Dropout(rate=0.5)(X)
    X = layers.Dense(128, activation = 'relu')(X)
    X = layers.Dropout(rate=0.5)(X)
    X = layers.Dense(1)(X)
    X = layers.Activation('sigmoid', dtype='float32', name='predictions')(X)
    model = models.Model(inputs = inputs, outputs = X)
    model.compile(optimizer = Adam(lr), loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
print(os.getcwd())

In [ ]:
for dataset,modelpath in zip(['/raid/audio_detection/audio_detection/data'],['/raid/audio_detection/audio_detection/seniors/data_0epochs 08_acc 0.9997.h5']):
    batch_size = 64
    target_size = (256, 256)
    color_mode = 'grayscale'
    val_datagen = ImageDataGenerator(rescale=1/255)
    val_generator = val_datagen.flow_from_directory(f'{dataset}/val/', target_size=target_size, batch_size=batch_size, class_mode='binary', color_mode=color_mode, classes = ['real','fake'])
    test_datagen = ImageDataGenerator(rescale=1/255)
    test_generator = test_datagen.flow_from_directory(f'{dataset}/test/', target_size=target_size, batch_size=batch_size, class_mode='binary', color_mode=color_mode, classes = ['real','fake'])
    model = small_cnn()
    model.load_weights(modelpath)
    print(f'Model: {modelpath}')
    results = model.evaluate(val_generator,steps=2*len(val_generator),verbose=0)
    print('Development Set:')
    print(f'Loss: {results[0]:.4f}\nAccuracy: {results[1]:.4f}')
    results = model.evaluate(test_generator,steps=2*len(test_generator),verbose=0)
    print('Evaluation Set:')
    print(f'Loss: {results[0]:.4f}\nAccuracy: {results[1]: .4f}')
    print()
